In [1]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

In [2]:

%pprint
%run ../../load_magic/storage.py
%run ../../load_magic/paths.py
%run ../../load_magic/lists.py
%run ../../load_magic/environment.py
%run ../../load_magic/dataframes.py

import scraping_utils
wikipedia = scraping_utils.wikipedia

from bs4 import BeautifulSoup as bs
import networkx as nx
import re
import urllib.request

notebook_path = get_notebook_path()
print(notebook_path)
s = Storage()
#pandemic_df = s.load_object('pandemic_df')
print(['s.{}'.format(fn) for fn in dir(s) if not fn.startswith('_')])
print(['wikipedia.{}'.format(fn) for fn in dir(wikipedia) if not fn.startswith('_')])

dir()

Pretty printing has been turned OFF
C:\Users\dev\Documents\repositories\notebooks\Miscellaneous\ipynb\US Events.ipynb
['s.attempt_to_pickle', 's.data_csv_folder', 's.data_folder', 's.encoding_type', 's.load_csv', 's.load_dataframes', 's.load_object', 's.save_dataframes', 's.saves_csv_folder', 's.saves_folder', 's.saves_pickle_folder', 's.store_objects']
['wikipedia.API_URL', 'wikipedia.BeautifulSoup', 'wikipedia.Decimal', 'wikipedia.DisambiguationError', 'wikipedia.HTTPTimeoutError', 'wikipedia.ODD_ERROR_MESSAGE', 'wikipedia.PageError', 'wikipedia.RATE_LIMIT', 'wikipedia.RATE_LIMIT_LAST_CALL', 'wikipedia.RATE_LIMIT_MIN_WAIT', 'wikipedia.RedirectError', 'wikipedia.USER_AGENT', 'wikipedia.WikipediaException', 'wikipedia.WikipediaPage', 'wikipedia.cache', 'wikipedia.datetime', 'wikipedia.debug', 'wikipedia.donate', 'wikipedia.exceptions', 'wikipedia.geosearch', 'wikipedia.languages', 'wikipedia.page', 'wikipedia.random', 'wikipedia.re', 'wikipedia.requests', 'wikipedia.search', 'wikipedia

['Config', 'In', 'Out', 'RandomForestClassifier', 'SequenceMatcher', 'Storage', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__nonzero__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i2', '_ih', '_ii', '_iii', '_oh', 'bs', 'check_4_doubles', 'check_for_typos', 'conjunctify_list', 'copyfile', 'csv', 'encoding', 'example_iterrows', 'exit', 'filepath_regex', 'get_classifier', 'get_column_descriptions', 'get_data_structs_dataframe', 'get_datastructure_prediction', 'get_dir_tree', 'get_git_lfs_track_commands', 'get_importances', 'get_input_sample', 'get_ipython', 'get_max_rsquared_adj', 'get_module_version', 'get_notebook_path', 'get_page_tables', 'get_specific_gitignore_files', 'get_struct_name', 'humanize_bytes', 'io', 'ipykernel', 'json', 'jupyter_config_dir', 'math', 'notebook_path', 'notebookapp', 'nx', 'os', 'pd', 'pickle', 'plt', 'preprocess_data', 'print_all_files_ending_starting_with', 'print_all_files_ending_with', 'print_all_files_st

In [3]:

tables_url = 'https://en.wikipedia.org/wiki/2020_in_the_United_States#Events_by_month'
tables_list = get_page_tables(tables_url)

[(0, (4, 4)), (4, (4, 2)), (5, (3, 2)), (1, (1, 4)), (6, (2, 2)), (3, (1, 2)), (2, (1, 1))]


In [201]:

page_obj = wikipedia.page(title='2020 in the United States')
#print(['page_obj.{}'.format(fn) for fn in dir(page_obj) if not fn.startswith('_')])


page_html = page_obj.html()
#print(['bs.{}'.format(fn) for fn in dir(bs) if not fn.startswith('_')])


page_soup = bs(page_html, 'html.parser')
#print(['page_soup.{}'.format(fn) for fn in dir(page_soup) if 'parent' in fn.lower()])


headline_soup = page_soup.select(selector='#Events_by_month')[0].parent
#print(['headline_soup.{}'.format(fn) for fn in dir(headline_soup) if 'next' in fn.lower()])

In [202]:

import nltk.data
from datetime import date

brackets_regex = re.compile(r'\[\d+\]\s*', re.MULTILINE)
dc_regex = re.compile(r'D\.C\.', re.MULTILINE)
us_regex = re.compile(r'U\.S\.', re.MULTILINE)
sent_regex = re.compile(r'\.([A-Z]|2020)', re.MULTILINE)
html_regex = re.compile(r'<[^>]+>', re.MULTILINE)
ws_regex = re.compile(r'\s+', re.MULTILINE)
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def clean_text(html_str):
    text_str = html_regex.sub(' ', html_str)
    text_str = ws_regex.sub(' ', text_str)
    text_str = brackets_regex.sub('', text_str)
    text_str = month_regex.sub('', text_str)
    text_str = dc_regex.sub(r'DC. ', text_str)
    text_str = us_regex.sub(r'US ', text_str)
    text_str = sent_regex.sub(r'. \1', text_str)
    
    return text_str

In [203]:

punct_regex = re.compile(r' (:|,|\.)')
sq_regex = re.compile(r" '(s| )")
sitat_regex = re.compile(r'\[ *citation needed *\] *')

def clean_sent(sent_str):
    sent_str = punct_regex.sub(r'\1', sent_str)
    sent_str = sq_regex.sub(r"'\1", sent_str)
    sent_str = sitat_regex.sub('', sent_str)
    
    return sent_str

In [204]:

month_dict = {}
text_dict = {}
for month_soup in headline_soup.find_next_siblings(name='h3')[0:10]:
    month_name = month_soup.select(selector='span.mw-headline')[0].text.strip()
    month_regex = re.compile(month_name + r'\s+\d+(–\d+)?:?\s+(–\s+)?', re.MULTILINE)
    month_set = set()
    content_soup = month_soup.find_next_siblings(name='ul')[0]
    for link in content_soup.select(selector='a'):
        attrs_dict = link.attrs
        if 'title' in attrs_dict:
            title = attrs_dict['title']
            if not title.startswith(month_name):
                month_set.add(title)
    month_dict[month_name] = sorted(list(month_set), key=lambda x: len(x))
    
    html_str = str(content_soup.extract())
    text_str = clean_text(html_str)
    sent_list = []
    for sent_str in tokenizer.tokenize(text_str):
        cleaned_str = clean_sent(sent_str.strip())
        sent_list.append(cleaned_str)
    text_dict[month_name] = sorted(sent_list, key=lambda x: len(x))

In [205]:

for month_name, event_list in text_dict.items():
    print()
    print(month_name)
    for event_name in event_list[-10:]:
        print(event_name)


January
Donald Trump becomes the first sitting president to personally attend the annual March for Life anti-abortion protest in Washington, DC.
2019–20 Persian Gulf crisis: Iraqi militiamen and protesters disperse from the site of the US  embassy in Baghdad a day after attacking it.
A helicopter crash in Calabasas, California kills nine people, including basketball star Kobe Bryant and his 13-year-old daughter Gianna Bryant.
Several new regulations take effect in the United States, including new regulations on retirement funds, new minimum wage rules, and new overtime rules.
COVID-19 pandemic: The first case of the COVID-19 coronavirus in the United States is confirmed by the Centers for Disease Control and Prevention (CDC).
The Women's National Basketball Association and its players' union reach an eight-year agreement allowing top players to earn $500,000 with an average salary of $130,000.
2019–20 Persian Gulf crisis: President Donald Trump approves the targeted killing of Iranian